# TODO Compared with from scratch standard training and attacks 

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
#TransferLearning of ATTACK --> 
# VS From Scratch
import torch
import torchvision
import torchvision.transforms as transforms
from utils import *
import torch.optim as optim
from model.baseline_model import *
from trainer import *
from torch.utils.tensorboard import SummaryWriter
import robustbench
#from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
from finetune_framework import *

import warnings
warnings.filterwarnings('ignore')

PyTorch Version:  1.7.1+cu110
Torchvision Version:  0.8.2+cu110


In [3]:
model_name = "resnet"
num_classes = 100
feature_extract = False

model_scratch, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)
train_dataset, val_dataset, train_loader, val_loader = CIFAR100(128, finetune = True, 
                                                               input_size = input_size, test_batch_size=128)
# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = "cuda"

0it [00:00, ?it/s]

Extracting ./datasets/CIFAR-100/cifar-100-python.tar.gz to ./datasets/CIFAR-100
Files already downloaded and verified


In [4]:
model_scratch = model_scratch.to(device)
params_to_update = model_scratch.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_scratch = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

In [6]:
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
PATH = './cifar_resnet_scratch_100.pth'
device = "cuda"
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("Standard_Resnet_Scratch_100")

In [ ]:
train(model_scratch, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_scratch, 100, writer, 
                 PATH, False, None, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	


In [ ]:
print("the clean accuracy is ")
test(val_loader, model_scratch)

In [ ]:
print("PGD")
pgd_attack_transfer = generate_adv(model_scratch, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, model_scratch, "cuda")

In [ ]:
print("fgsm")
fgsm_attack_transfer = generate_adv(model_scratch, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, model_scratch, "cuda")

In [ ]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(model_scratch, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, model_scratch, "cuda")

In [ ]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(model_scratch, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, model_scratch, "cuda")

# TODO Compared with from scratch adversarial training and attacks 

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_scratch_adv = copy.deepcopy(model_scratch) #am I doing correct? 
# for param in net_adv.parameters():
#     print(param)

In [ ]:
params_to_update = net_scratch_adv.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in net_scratch_adv.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_adv = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()

PATH = './cifar_trans_resnet_scratch_adv_pgd.pth'
device = "cuda"
writer = SummaryWriter("Adv_trans_resnet_scratch_PGD")
pgd_attack = generate_adv(net_scratch_adv, "pgd")

In [ ]:
train(net_scratch_adv, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_adv, 100, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

In [ ]:
print("the clean accuracy is ")
test(val_loader, net_scratch_adv)

In [ ]:
print("PGD")
pgd_attack_transfer = generate_adv(model_scratch, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, net_scratch_adv, "cuda")

In [ ]:
print("fgsm")
fgsm_attack_transfer = generate_adv(model_scratch, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, net_scratch_adv, "cuda")

In [ ]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(model_scratch, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, net_scratch_adv, "cuda")

In [ ]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(model_scratch, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, net_scratch_adv, "cuda")